# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 9:55AM,250142,10,8745051-SD,Eywa Pharma Inc.,Released
1,Nov 7 2022 9:47AM,250141,10,PRF-41047,Bio-PRF,Released
2,Nov 7 2022 9:47AM,250141,10,PRF-41050,Bio-PRF,Released
3,Nov 7 2022 9:47AM,250141,10,PRF-41053,Bio-PRF,Released
4,Nov 7 2022 9:47AM,250141,10,PRF-41056,Bio-PRF,Released
5,Nov 7 2022 9:47AM,250141,10,PRF-41060,Bio-PRF,Released
6,Nov 7 2022 9:47AM,250141,10,PRF-41063,Bio-PRF,Released
7,Nov 7 2022 9:47AM,250141,10,PRF-41069,Bio-PRF,Released
8,Nov 7 2022 9:47AM,250141,10,PRF-41077,Bio-PRF,Released
9,Nov 7 2022 9:47AM,250141,10,PRF-41080,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,250131,Released,1
9,250132,Released,1
10,250140,Released,1
11,250141,Released,30
12,250142,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250131,NaN,1.0
250132,NaN,1.0
250140,NaN,1.0
250141,NaN,30.0
250142,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250118,2.0,4.0
250121,0.0,1.0
250122,12.0,0.0
250124,0.0,62.0
250125,0.0,59.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250118,2,4
250121,0,1
250122,12,0
250124,0,62
250125,0,59


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250118,2,4
1,250121,0,1
2,250122,12,0
3,250124,0,62
4,250125,0,59


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250118,2,4
1,250121,,1
2,250122,12,
3,250124,,62
4,250125,,59


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF
31,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.
32,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.
33,Nov 7 2022 9:29AM,250131,18,Seqirus
34,Nov 7 2022 9:29AM,250130,16,Seqirus
35,Nov 7 2022 9:18AM,250129,20,"HVL, LLC dba Atrium Innovations"
36,Nov 7 2022 9:08AM,250125,10,ISDIN Corporation
95,Nov 7 2022 9:08AM,250124,10,ISDIN Corporation
157,Nov 7 2022 8:47AM,250122,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.,,1
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF,,30
2,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.,,1
3,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.,,1
4,Nov 7 2022 9:29AM,250131,18,Seqirus,,1
5,Nov 7 2022 9:29AM,250130,16,Seqirus,,1
6,Nov 7 2022 9:18AM,250129,20,"HVL, LLC dba Atrium Innovations",,1
7,Nov 7 2022 9:08AM,250125,10,ISDIN Corporation,,59
8,Nov 7 2022 9:08AM,250124,10,ISDIN Corporation,,62
9,Nov 7 2022 8:47AM,250122,10,ISDIN Corporation,12,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.,1,
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF,30,
2,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.,1,
3,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.,1,
4,Nov 7 2022 9:29AM,250131,18,Seqirus,1,
5,Nov 7 2022 9:29AM,250130,16,Seqirus,1,
6,Nov 7 2022 9:18AM,250129,20,"HVL, LLC dba Atrium Innovations",1,
7,Nov 7 2022 9:08AM,250125,10,ISDIN Corporation,59,
8,Nov 7 2022 9:08AM,250124,10,ISDIN Corporation,62,
9,Nov 7 2022 8:47AM,250122,10,ISDIN Corporation,,12


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.,1,
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF,30,
2,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.,1,
3,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.,1,
4,Nov 7 2022 9:29AM,250131,18,Seqirus,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.,1.0,NaN
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF,30.0,NaN
2,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.,1.0,NaN
3,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.,1.0,NaN
4,Nov 7 2022 9:29AM,250131,18,Seqirus,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 9:55AM,250142,10,Eywa Pharma Inc.,1.0,0.0
1,Nov 7 2022 9:47AM,250141,10,Bio-PRF,30.0,0.0
2,Nov 7 2022 9:41AM,250140,20,Alumier Labs Inc.,1.0,0.0
3,Nov 7 2022 9:31AM,250132,20,Alumier Labs Inc.,1.0,0.0
4,Nov 7 2022 9:29AM,250131,18,Seqirus,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1250654,152.0,12.0
16,500251,2.0,0.0
18,250131,1.0,0.0
19,250118,4.0,2.0
20,750401,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1250654,152.0,12.0
1,16,500251,2.0,0.0
2,18,250131,1.0,0.0
3,19,250118,4.0,2.0
4,20,750401,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,152.0,12.0
1,16,2.0,0.0
2,18,1.0,0.0
3,19,4.0,2.0
4,20,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,152.0
1,16,Released,2.0
2,18,Released,1.0
3,19,Released,4.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Executing,12.0,0.0,0.0,2.0,0.0
Released,152.0,2.0,1.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Executing,12.0,0.0,0.0,2.0,0.0
1,Released,152.0,2.0,1.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Executing,12.0,0.0,0.0,2.0,0.0
1,Released,152.0,2.0,1.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()